<a href="https://colab.research.google.com/github/toddlack/colab/blob/main/coquiTTS_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/drive')

Copy files from google drive to session's  /content folder

In [ ]:
from google.colab import files
%mkdir -p /content/tts_models/tammy09apr
%cp /drive/MyDrive/ai_voice/tammy09apr-300epochs/tammy09apr-300epochs9300.pth /content/tts_models/tammy09apr

Install Dependencies

In [ ]:
# @title Clone repo for local use
#Clone a repo and cd into the new directory
content_root = "/content" # @param {"type":"string","placeholder":"content root"}
git_repo='https://github.com/coqui-ai/TTS.git'  # @param {"type":"string","placeholder":"git repo url"}
git_branch='dev'  # @param {"type":"string","placeholder":"git branch"}
app_root=git_repo.split('/')[-1].replace('.git', '')
assets_dir=f'{content_root}/{app_root}/assets'
%cd {content_root}
!git clone --single-branch -b {git_branch} {git_repo}
%cd {app_root}

Copy dataset (audio) to train

In [ ]:
# prompt: copy directory from dataset_source variable to content_root/dataset

# Copy dataset (audio) to train
dataset_source = '/drive/MyDrive/ai_voice/training_audio/tammy' # @param {"type":"string","placeholder":"path to source audio dataset"}
dataset_destination = 'dataset/audio' # @param {"type":"string","placeholder":"path to audio destination"}
%mkdir -p "{content_root}/{dataset_destination}"
%cp -rv "{dataset_source}" "{content_root}/{dataset_destination}"

In [ ]:
# prompt: python code to generate text to voice using coqui TTS and pytorch
%pip install numpy==1.22 # Install the required NumPy version
%pip install TTS --upgrade --no-cache-dir # Reinstall/upgrade TTS after fixing NumPy, to ensure it links against the correct NumPy version
%pip install torch



Set TTS

In [ ]:
import torch
from TTS.api import TTS

#Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path_tammy="/content/tts_models/tammy09apr/"
config_path_tammy="/content/tts_models/tammy09apr/config.json"

# Init TTS
tts = TTS("tts_models/en/ljspeech/tacotron2-DDC")  # Load the desired model
ttsTammy = TTS(model_path_tammy, config_path_tammy)  # Load the desired model
# Run TTS
tts.tts_to_file(text="Hello world!", speaker=None, file_path="output.wav")
ttsT.tts_to_file(text="Hello world!", speaker=None, file_path="outputTammy.wav")
